<a href="https://colab.research.google.com/github/skywalker0803r/oil_phase2/blob/master/%E6%A0%B8%E5%BF%83%E7%B5%84%E6%88%90%E9%A0%90%E6%B8%AC%E6%A8%A1%E7%B5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import warnings;warnings.simplefilter('ignore')

df = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/phase2/解壓過的/train.csv',index_col=0)

# 處理一下
df = df.dropna()
df = df.apply(lambda x:round(x,2))

print(df.shape)
df.head()

(14023, 274)


,Case Conditions_Feed Rate to C013 (Input),Case Conditions_C7+ in Light End (Input),Case Conditions_Heart Cut Prod. Rate (Input),Case Conditions_C6P- in Heart Cut (Input),Case Conditions_Feed Rate to C013 (Results),Case Conditions_C7+ in Light End (Results),Case Conditions_Heart Cut Prod. Rate (Results),Case Conditions_ C6P- in Heart Cut (Results),Case Conditions_ C10+ in Heart Cut (Results),Naphtha Properties_Naphtha Feed Rate,Naphtha Properties_Naphtha Feed Rate.1,Naphtha Properties_ N+A in Naphtha,Naphtha Properties_ C6P- in Naphtha,Naphtha Properties_ C10+ in Naphtha,Naphtha Properties_ Oxygen,Naphtha Properties_ Methane,Naphtha Properties_ Ethane,Naphtha Properties_ Propane,Naphtha Properties_ n-Butane,Naphtha Properties_ i-Pentane,Naphtha Properties_ n-Pentane,Naphtha Properties_ tr2-Pentene,Naphtha Properties_ Cyclopentane,Naphtha Properties_ 3-Methylpentane,Naphtha Properties_ tr2-Hexene,Naphtha Properties_ n-Hexane,Naphtha Properties_ Methylcyclopentane,"Naphtha Properties_ 2,2-Dimethylpentane",Naphtha Properties_ Benzene,Naphtha Properties_ Cyclohexane,"Naphtha Properties_ 2,2,3-Trimethylbutane",Naphtha Properties_ 4-Methyl-1-hexene,Naphtha Properties_ 2-Methylhexane,"Naphtha Properties_ t-1,3-Dimethylcyclopentane",Naphtha Properties_ n-Heptane,Naphtha Properties_ H2O,Naphtha Properties_ Methylcyclohexane,Naphtha Properties_ Ethylcyclopentane,"Naphtha Properties_ 2,3-Dimethyl-1-hexene",Naphtha Properties_ Toluene,...,Heavy End Product Properties_ Cyclohexane,"Heavy End Product Properties_ 2,2,3-Trimethylbutane",Heavy End Product Properties_ 4-Methyl-1-hexene,Heavy End Product Properties_ 2-Methylhexane,"Heavy End Product Properties_ t-1,3-Dimethylcyclopentane",Heavy End Product Properties_ n-Heptane,Heavy End Product Properties_ H2O,Heavy End Product Properties_ Methylcyclohexane,Heavy End Product Properties_ Ethylcyclopentane,"Heavy End Product Properties_ 2,3-Dimethyl-1-hexene",Heavy End Product Properties_ Toluene,Heavy End Product Properties_ 2-Methylheptane,Heavy End Product Properties_ n-Octane,Heavy End Product Properties_ Ethylcyclohexane,Heavy End Product Properties_ EthylBenzene,Heavy End Product Properties_ P-Xylene,Heavy End Product Properties_ M-Xylene,Heavy End Product Properties_ O-Xylene,Heavy End Product Properties_ 2-Methyloctane,Heavy End Product Properties_ 1-Nonene,Heavy End Product Properties_ n-Nonane,Heavy End Product Properties_ i-Propylbenzene,"Heavy End Product Properties_ 1-trans-3,5-Trimethylcyclohexane",Heavy End Product Properties_ n-Propylcyclohexane,Heavy End Product Properties_ n-Propylbenzene,"Heavy End Product Properties_ 1,3,5-Trimethylbenzene",Heavy End Product Properties_ 1-Methyl-2-ethylbenzene,"Heavy End Product Properties_ 1,2,3-Trimethylbenzene","Heavy End Product Properties_ 2,4-Dimethyloctane","Heavy End Product Properties_ 2,2,3,3-Tetramethylhexane",Heavy End Product Properties_ 3-Methylnonane,Heavy End Product Properties_ tert-Butylcyclohexane,Heavy End Product Properties_ n-Decane,"Heavy End Product Properties_ 1,2-Diethylbenzene",Heavy End Product Properties_ n-Undecane,Heavy End Product Properties_ n-Pentylbenzene,Heavy End Product Properties_ n-Dodecane,Heavy End Product Properties_ n-Hexylbenzene,Heavy End Product Properties_ n-Tridecane,Heavy End Product Properties_Overall
0,10.0,2.5,100.0,1.0,10.0,2.5,100.0,1.0,6.14,118.0,86.48,30.19,5.78,8.34,0.0,0.0,0.0,0.0,0.0,0.03,0.07,0.0,0.01,1.37,0.03,4.27,1.5,0.74,0.4,0.81,0.82,0.02,6.18,0.55,10.8,0.0,3.2,2.21,0.11,2.86,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.15,0.02,0.04,0.13,0.67,17.20,0.72,22.29,0.16,3.15,6.53,0.77,3.58,4.82,4.18,3.46,5.50,13.96,2.50,5.28,3.51,1.09,0.22,0.02,0.04,0.0,100.0
1,10.0,5.0,100.0,1.0,10.0,5.0,100.0,1.0,6.25,118.0,86.48,30.19,5.78,8.34,0.0,0.0,0.0,0.0,0.0,0.03,0.07,0.0,0.01,1.37,0.03,4.27,1.5,0.74,0.4,0.81,0.82,0.02,6.18,0.55,10.8,0.0,3.2,2.21,0.11,2.86,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.14,0.02,0.04,0.11,0.61,16.59,0.72,22.41,0.16,2.97,6.59,0.78,3.63,

# columns

In [2]:
prefix = sorted(list(set([i.split('_')[0] for i in df.columns])))
for i in prefix:
  print(i)

Case Conditions
Column Temp Profile
Duty
Feed
Heart Cut Product Properties
Heavy End Product Properties
Light End Product Properties
Naphtha Properties
Operating Pressure
Specifications
Yield Summary


In [0]:
col_group = {}
for i in prefix:
  col_group[i] = df.columns[df.columns.str.contains(i)].tolist()

# data prepare

In [4]:
# 核心輕油成份
y_col = col_group['Heart Cut Product Properties']
# 產品規格 進料組成
x_col = col_group['Specifications'] + col_group['Naphtha Properties'] + col_group['Column Temp Profile']

X = df[x_col]
y = df[y_col]
print(X.shape)
print(y.shape)

# feature scale
TR = StandardScaler()
X[:] = TR.fit_transform(X[:])

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

(14023, 75)
(14023, 59)


# build model

In [0]:
model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors = 5,
                                                 n_jobs = -1))

# fit

In [6]:
model.fit(X_train,y_train)

MultiOutputRegressor(estimator=KNeighborsRegressor(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=-1, n_neighbors=5,
                                                   p=2, weights='uniform'),
                     n_jobs=None)

# predict

In [0]:
y_pred = y_test.copy()
y_pred[:] = model.predict(X_test)

# evaluate

In [0]:
res = pd.DataFrame(index=y_col,columns=['R2','MSE'])
for i in y_col:
  res.loc[i,'R2'] = r2_score(y_test[i],y_pred[i])
  res.loc[i,'MSE'] = mean_squared_error(y_test[i],y_pred[i])
res.loc['AVG'] = res.mean(axis=0)

In [9]:
res

,R2,MSE
Heart Cut Product Properties_Heart Cut Prod. Rate,0.876383,2.21954
Heart Cut Product Properties_Heart Cut Prod. Rate.1,0.889068,1.26955
Heart Cut Product Properties_ C6P- in Heart Cut,0.932105,0.0198531
Heart Cut Product Properties_ C10+ in Heart Cut,0.981815,0.0614805
Heart Cut Product Properties_ Oxygen,1,0
Heart Cut Product Properties_ Methane,1,0
Heart Cut Product Properties_ Ethane,1,0
Heart Cut Product Properties_ Propane,1,0
Heart Cut Product Properties_ n-Butane,0.420904,2.93691e-06
Heart Cut Product Properties_ i-Pentane,0.828557,0.000115685
